In [75]:
# Installations
!pip install gzip
!pip install geopy
!pip install boto

In [1]:
#jsonlines code
# Based on:
import jsonlines
import pprint
import json
import numpy as np
import pandas as pd
import gzip
import os

In [ ]:
#Get all relevant folders across all subdirectories
dir_name = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test'
extension = ".gz"

#os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        print(item)
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [4]:
file = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-01-21-22.jsonl.gz'
f = gzip.open(file, 'rb')
file_content = f.readlines()
f.close()

#Create list of dictionaries
final_json = [json.loads(line.decode('utf8')) for line in file_content]


RT @AnneKPIX: @CDC has activated its emergency operations center. 
They expect more US cases.
#coronavirus


In [5]:
link = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-03-08-17.jsonl'
json_list =[]
with jsonlines.open(link) as f:
    for line in f.iter():
        try:
            del line['entities']
        except:
            pass            
            
        try:
            del line['retweeted_status']
        except:
            #since some tweets were never retweeted
            pass
        #we don't want unnecessary multi-level nesting
        del line['user']['entities']        
        json_list.append(line)

## Understanding the Shape of the Data

In [9]:
test = json_list[3]
pprint.pprint(test)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Mar 08 17:02:34 +0000 2020',
 'display_text_range': [0, 140],
 'favorite_count': 0,
 'favorited': False,
 'full_text': 'RT @RinChupeco: Given coronavirus fears, more people should '
              'know about the Corrupted Blood bug in World of Warcraft that '
              'caused player char…',
 'geo': None,
 'id': 1236698840373383200,
 'id_str': '1236698840373383168',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 8654,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" '
           'rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'user': {'contributors_enabled': False,
          'created_at': 'Fri Apr 18 16:15:45 +0000 2008',
          'default_profile': False,
          'default_profile

In [10]:
cols = list(test.keys())
cols.remove('user')
print(len(cols))
print(cols)

22
['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang']


In [ ]:
#normalizing json
# https://stackoverflow.com/questions/40588852/pandas-read-nested-json
from pandas.io.json import json_normalize 
df_normal = json_normalize(json_list,
                           record_path=['user'],
                           meta=cols,
                           #record_prefix='user_',
                           #meta_prefix='tweet_',
                           errors='ignore')

In [14]:
len(df_normal.columns)

NameError: name 'df_normal' is not defined

So that's the culprit- entities has a 3-level nested structure. That's what's getting in the way of normalization

Get rid of the entities key in the original json, as opposed to  
https://stackoverflow.com/questions/46599519/how-to-remove-json-object-key-and-value/46599555

In [ ]:
del cols['entities']

In [7]:
#https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries
def pretty(d, indent=2):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [8]:
pretty(test)

		created_at
			Sun Mar 08 17:02:34 +0000 2020
		id
			1236698840373383200
		id_str
			1236698840373383168
		full_text
			RT @RinChupeco: Given coronavirus fears, more people should know about the Corrupted Blood bug in World of Warcraft that caused player char…
		truncated
			False
		display_text_range
			[0, 140]
		source
			<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
		in_reply_to_status_id
			None
		in_reply_to_status_id_str
			None
		in_reply_to_user_id
			None
		in_reply_to_user_id_str
			None
		in_reply_to_screen_name
			None
		user
			id
				14434693
			id_str
				14434693
			name
				WilliamOwen, True Believer
			screen_name
				William_A_Owen
			location
				Brooklyn
			description
				Novelist, Comic Creator and Flash Fictioneer. Once and Future Bumpkin. Arbiter of Pie. Digital chicanery at Penguin Random House. KHive. Opinions are my own.
			url
				http://t.co/hGOwcLYh
			entities
				url
					urls
						[{'url': 'http://t.co/hGOwcLYh

In [10]:
test['retweeted_status']

{'created_at': 'Sat Mar 07 23:56:32 +0000 2020',
 'id': 1236440629258801200,
 'id_str': '1236440629258801152',
 'full_text': 'Given coronavirus fears, more people should know about the Corrupted Blood bug in World of Warcraft that caused player chars to be infected with a killing debuff that soon spread to other players\n\nEpidemiologists actually STUDIED this to see how ppl would react to an epidemic irl',
 'truncated': False,
 'display_text_range': [0, 280],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 92699184,
  'id_str': '92699184',
  'name': 'Rin Chupeco',
  'screen_name': 'RinChupeco',
  'location': 'Manila, Philippines',
  'description': '🇵🇭 cantankerous old man in an 

## Understanding Data Structure

Some things are clear here.
At the base level, there's information about the tweet
Then comes data on the user- name, description and URL. Here, the entities key contains urls an descriptions (nested). 
A few more fields for the tweet itself- geo related. 




In [ ]:
import json
parsed = json.loads(cols)
print(json.dumps(parsed, indent=4))

In [ ]:
#check old items
json_list[48]

Ahhhh so the JSON is deeply nested, which is why we ended up with only 32 columns. The remaining ones were eaten up/ subsumed in one column.Now we need to pick the ones that actually matter and ignore the rest. 



https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal
What does the 'b' mean when I try unpacking the data?
So it's actually a binary file. Then it should work with jsonlines. Let's try again

In [ ]:
file = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-01-21-22.jsonl.gz'
with jsonlines.open(gzip.open(file, 'rb'))
as f.read():
        for line in f.iter():
            print(line['full_text'])

The problem appears to be in the location of the file:
https://cbsetoday.com/text-binary-file-handling-python/


First, we need to unzip all the files. They are kept in the .gz files. 
https://stackoverflow.com/questions/31346790/unzip-all-zipped-files-in-a-folder-to-that-same-folder-using-python-2-7-5


# Analyzing Tweet Content

In [77]:
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string

# Some Functions from Last Time to get us started:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}

    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

def get_lemmas(text):
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i for i in nltk.word_tokenize(text.lower()) if i not in stop]
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(t, get_wordnet_pos(t)) for t in tokens]
    return lemmas

def get_tokens(text):
    # drop punctuation, but keep stopwords for initial word counting
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [i for i in nltk.word_tokenize(text.lower())]
    return tokens

def fill_topic_weights(df_row, bow_corpus):
    # Fill topic weights for topics in songs
    try:
        for i in ldamodel[bow_corpus[df_row.name]]:
            df_row[str(i[0])] = i[1]
    except:
        return df_row
    return df_row


In [ ]:
# Topic Modelling

def top_songs_by_topic(ldamodel, corpus, ntop=1):
    #Returns empty dictionary
    topn_songs_by_topic = {}
    for i in range(len(ldamodel.print_topics())):
        # For each topic, collect the most representative song(s) (i.e. highest probability containing words belonging to topic):
        top = sorted(zip(range(len(corpus)), ldamodel[corpus]), reverse=True, key=lambda x: abs(dict(x[1]).get(i, 0.0)))
        topn_songs_by_topic[i] = [j[0] for j in top[:ntop]]
        # Print out the topn songs for each topic and return their indices as a dictionary for further analysis:
        print("Topic " + str(i))
        print(music_df[['title','year','artist']].loc[topn_songs_by_topic[i]])
        print("*******************************")
    return topn_songs_by_topic

In [84]:
df = pd.DataFrame(json_list)
df_eng = df[df['lang']=='en']
tweet_text = df_eng['full_text'].apply(get_tokens)


In [85]:
tweet_text

0        [rt, thedailybeast, snl, brutally, roasts, tru...
1        [rt, mlipsitch, important, evidence, on, role,...
3        [rt, rinchupeco, given, coronavirus, fears, mo...
5        [rt, briankarem, why, are, they, lying, to, us...
6        [rt, cameronlmitchel, kylegriffin1, the, trump...
9        [white, house, continues, to, lie, about, coro...
10       [rt, ketengahketepi, maaf, tanya, part, happy,...
11       [rt, granttucker, britain, we, survived, the, ...
12       [rt, atrupar, the, president, is, golfing, at,...
13       [rt, khjesq, coronavirus, got, boomers, stocki...
14       [rt, markjacob16, disappointed, in, the, consp...
15       [rt, mikepence, as, a, result, of, president, ...
16       [rt, anthonydragons, aiiight, that, ’, s, it, ...
19       [rt, mollyjongfast, the, level, of, incompeten...
22       [rt, fairyglowmother, important, to, note, tha...
25       [surgeongeneral, indeed, you, repeated, this, ...
26       [rt, robmose, wow, a, nursing, home, in, seatt.

In [86]:
#Measure religiosity
def relig_perc(text):
    relig_count = text.count("God") + text.count("Jesus") + text.count("church") + text.count("Lord") + text.count("pray") + text.count("faith")
    return relig_count/len(text)

In [87]:
religiosity = tweet_text.apply(relig_perc)

In [91]:
print(religiosity.max())

0.2


# Extracting the Geocodes

In [ ]:
from geopy.geocoders import OpenMapQuest
from geopy.extra.rate_limiter import RateLimiter
geolocator = OpenMapQuest(api_key='FGtjwBYCTEadGZpAIEkM2ySw5518bS7b') 
geocoder = RateLimiter(geolocator.geocode, min_delay_seconds=1) 
fwp_df['location'] = fwp_df['userlocation'].apply(geocoder)